#### FUNDAÇÃO GETULIO VARGAS
#### ESCOLA BRASILEIRA DE ECONOMIA E FINANÇAS
#### CURSO DE GRADUAÇÃO EM CIÊNCIAS ECONÔMICAS 
##### ANÁLISE COMPARATIVA ENTRE INVESTIMENTO DIRETO EM IMÓVEIS RESIDENCIAIS E FUNDOS DE INVESTIMENTO IMOBILIÁRIO
###### Deivid Cezar da Silva
###### Rio de Janeiro, 2021


In [1]:
# Importando bibliotecas
import pandas as pd
import altair as alt
from numpy import cov
import statsmodels.api as sm
alt.renderers.enable('default')

# Importando dados coletados do boletim mensal da B3 de agosto de 2021
df_b3_nf = pd.read_excel('boletimb3fiis.xlsx', sheet_name = 'Número de FIIS listados')
df_b3_ni = pd.read_excel('boletimb3fiis.xlsx', sheet_name = 'Número de investidores')
df_b3_vp = pd.read_excel('boletimb3fiis.xlsx', sheet_name = 'PVP')

# Bases de dados do portal FipeZap
df_fz = pd.read_excel('fipezap-serieshistoricas.xlsx', sheet_name = 'Índice FipeZap')
df_rj = pd.read_excel('fipezap-serieshistoricas.xlsx', sheet_name = 'Rio de Janeiro')
df_sp = pd.read_excel('fipezap-serieshistoricas.xlsx', sheet_name = 'São Paulo')

# Informações retiradas do relatório Santander Equity Research (Brasil - Fundos Imobiliários) de janeiro de 2021
df_composicao2011 = pd.read_excel('santanderequityresearch.xlsx', sheet_name = 'Composição 2011')
df_composicao2021 = pd.read_excel('santanderequityresearch.xlsx', sheet_name = 'Composição 2021')
df_ifixyield = pd.read_excel('santanderequityresearch.xlsx', sheet_name = 'Yield')

# Demais dados
df_t10y = pd.read_excel('economatica_realtreasury10yrs.xlsx', skiprows = 3)
df_sp500 = pd.read_excel('economatica_sp500.xlsx', skiprows = 3)
df_ifix = pd.read_excel('evolucao_mensal.xlsx', skiprows = 1)
df_cdi = pd.read_excel('economatica_cdi.xlsx', skiprows = 3)
df_ipca = pd.read_excel('seriehistoricaipca.xlsx')
df_embi = pd.read_excel('embi+.xlsx')

#### 1. Evolução do número de investidores com posição em custódia de fundos imobiliários.

In [2]:
# Plotando gráfico de barras com o número de investidores em fundos imobiliários
base = alt.Chart(df_b3_ni).mark_bar().encode(
    x = alt.X('yearmonth(Data):O', title = 'Ano'),
    y = alt.Y('Número de investidores (mil):Q'),
    color = alt.value('#110799')
).properties(width = 640, height = 360, 
             title = 'Evolução do número de investidores com posição em custódia de fundos imobiliários')

text = base.mark_text(baseline = 'bottom').encode(
    text = alt.Text('Número de investidores (mil):Q'),
    color = alt.value('#000000'))

(base + text).configure(background = '#EEE')

alt.LayerChart(...)

#### 2. Evolução do número de fundos imobiliários listados.

In [3]:
# Plotando gráfico de barras com a quantidade de fundos listados
base = alt.Chart(df_b3_nf).mark_bar().encode(
    x = alt.X('yearmonth(Data):O', title = 'Ano'),
    y = alt.Y('Número de FIIS listados:Q', title = 'Número de FIIs'),
    color = alt.value('#110799')
).properties(width = 640, height = 360, title = 'Evolução do número de fundos imobiliários listados')

text = base.mark_text(baseline = 'bottom').encode(
    text = alt.Text('Número de FIIS listados:Q'),
    color = alt.value('#000000'))

(base + text).configure(background = '#EEE')

alt.LayerChart(...)

#### 3. Evolução do valor de mercado e do patrimônio líquido dos fundos imobiliários.

In [4]:
# Plotando gráfico de linhas com comparativo entre o valor de mercado e patrimônio líquido dos fundos imobiliários
base = alt.Chart(df_b3_vp.melt('Data', var_name = 'Indicador', value_name = 'Bilhões de reais')).mark_line(point = True).encode(
    x = alt.X('yearmonth(Data):O', title = 'Mês'),
    y = alt.Y('Bilhões de reais:Q'),
    color = alt.Color('Indicador', 
                      scale = alt.Scale(domain = ['Valor de mercado', 'Patrimônio líquido'], range = ['#110799', '#8F9907']))
).properties(width = 640, height = 360, title = 'Evolução do valor de mercado e do patrimônio líquido dos fundos imobiliários')

text = base.mark_text(baseline = 'bottom', dy = -2.5).encode(
    text = alt.Text('Bilhões de reais:Q'),
    color = alt.value('#000000'))

(base + text).configure(background = '#EEE')

alt.LayerChart(...)

#### 4. Evolução do preço sobre valor patrimonial dos fundos imobiliários.

In [5]:
# Calculando preço / valor patrimonial
df_b3_vp['P/VP'] = df_b3_vp['Valor de mercado'] / df_b3_vp['Patrimônio líquido']

# Limitando a quantidade de casas depois da vírgula
df_b3_vp['P/VP'] = round(df_b3_vp['P/VP'], 2)

# Plotando gráfico de linhas com o indicador e sua média
base = alt.Chart(df_b3_vp).mark_line(point = True).encode(
    x = alt.X('yearmonth(Data):O', title = 'Mês'),
    y = alt.Y('P/VP:Q', title = 'P/VP e média', scale = alt.Scale(domain = [0.75, 1.25])),
    color = alt.value('#110799')
).properties(width = 640, height = 360, title = 'Evolução do preço sobre valor patrimonial dos fundos imobiliários')

line = alt.Chart(df_b3_vp).mark_rule().encode(
    y = alt.Y('mean(P/VP):Q', title = ''),
    color = alt.value('#8F9907'))

text = base.mark_text(baseline = 'bottom', dy = -5).encode(
    text = alt.Text('P/VP:Q'),
    color = alt.value('#000000'))

(base + line + text).configure(background = '#EEE')

alt.LayerChart(...)

#### 5. Desempenho histórico do Índice de Fundos de Investimento Imobiliário (IFIX).

In [6]:
# Convertendo os dados de meses e anos para strings
df_ifix[['Mês','Ano']] = df_ifix[['Mês','Ano']].astype(str)

# Agrupando colunas de meses e anos
df_ifix['Data'] = df_ifix['Mês'] + '/' + df_ifix['Ano']

# Removendo colunas de meses e anos isolados
df_ifix = df_ifix.drop(['Mês','Ano'], axis = 1)

# Convertendo coluna de datas para datetime
df_ifix['Data'] = pd.to_datetime(df_ifix['Data'])

# Plotando gráfico de linha do desempenho histórico do IFIX
alt.Chart(df_ifix).mark_line().encode(
    x = 'Data:T',
    y = 'Valor',
    color = alt.value('#110799')
).properties(width = 640, height = 360, title = 'Desempenho histórico do Índice de Fundos de Investimento Imobiliário (IFIX)',
             background = '#EEE')

alt.Chart(...)

#### 6. Calculando o retorno mensal do índice.

In [7]:
# Copiando data frame anterior
df_ifix_retorno = df_ifix.copy()

# Calculando retorno
df_ifix_retorno['Retorno mensal do IFIX'] = df_ifix_retorno['Valor'] / df_ifix_retorno['Valor'].shift(1) - 1

# Recortando primeira linha e coluna de valores do índice
df_ifix_retorno = df_ifix_retorno.iloc[1:, 1:].reset_index(drop = True)

# Plotando gráfico heatmap da variação ao longo do tempo
base = alt.Chart(df_ifix_retorno).mark_rect().encode(
    alt.X('month(Data):O', title = 'Mês'),
    alt.Y('year(Data):O', title = 'Ano'),
    alt.Color('Retorno mensal do IFIX:Q', title = 'Variação', scale = alt.Scale(scheme = 'redblue', domain = [-0.2, 0.2]), 
              legend = alt.Legend(format = '.0%'))
).properties(width = 640, height = 360, title = 'Variação mensal do IFIX')

text = base.mark_text(baseline = 'middle').encode(
    text = alt.Text('Retorno mensal do IFIX:Q', format = '.1%'),
    color = alt.value('#000000'))

base + text

alt.LayerChart(...)

#### 7. Dividend yield do IFIX.

In [8]:
# Limitando a quantidade de casas depois da vírgula na coluna de dividend yield em zero (valor aproximado)
df_ifixyield['Dividend yield text'] = ((df_ifixyield['Dividend yield'] * 100).astype(int).astype(str) + '%')

# Plotando gráfico de barras com o dividend yield anual médio do IFIX
base = alt.Chart(df_ifixyield).mark_bar().encode(
    x = alt.X('Ano:O'),
    y = alt.Y('Dividend yield:Q', title = 'Dividend yield e média', axis = alt.Axis(format = '%'), 
              scale = alt.Scale(domain = [0, 0.135])),
    color = alt.value('#110799')
).properties(width = 640, height = 360, title = 'Dividend yield anual médio do IFIX')

line = alt.Chart(df_ifixyield).mark_rule().encode(
    y = alt.Y('mean(Dividend yield):Q', title = ''),
    color = alt.value('#8F9907'))

text = base.mark_text(baseline = 'bottom').encode(
    text = alt.Text('Dividend yield text:N'),
    color = alt.value('#000000'))

(base + line + text).configure(background = '#EEE')

alt.LayerChart(...)

#### 8. Evolução do preço do metro quadrado dos imóveis residenciais do índice FipeZap geral, em São Paulo e no Rio de Janeiro.

In [9]:
# Selecionando dados do preço médio do metro quadrado de imóveis residenciais para venda
df_fz_vendas = df_fz.iloc[38:166, [1,17]]
df_sp_vendas = df_sp.iloc[38:166, [1,17]]
df_rj_vendas = df_rj.iloc[38:166, [1,17]]

# Renomeando colunas
df_fz_vendas.columns = ['Data', 'Preço médio (R$/m^2)']
df_sp_vendas.columns = ['Data', 'Preço médio (R$/m^2)']
df_rj_vendas.columns = ['Data', 'Preço médio (R$/m^2)']

# Convertendo coluna de datas para datetime
df_fz_vendas['Data'] = pd.to_datetime(df_fz_vendas['Data'])
df_sp_vendas['Data'] = pd.to_datetime(df_sp_vendas['Data'])
df_rj_vendas['Data'] = pd.to_datetime(df_rj_vendas['Data'])

# Organizando indexes
df_fz_vendas = df_fz_vendas.reset_index(drop = True)
df_sp_vendas = df_sp_vendas.reset_index(drop = True)
df_rj_vendas = df_rj_vendas.reset_index(drop = True)

# Função para construção do gráfico de linha do preço do metro quadrado dos imóveis residenciais ao longo do tempo
def precometroquadrado(df_pmq, title_pmq):
    
    chart_pmq = alt.Chart(df_pmq).mark_line().encode(
        x = 'Data:T',
        y = alt.Y('Preço médio (R$/m^2)', scale = alt.Scale(domain = [0, 12000])),
        color = alt.value('#110799')
    ).properties(width = 640, height = 360, 
                 title = 'Evolução do preço do metro quadrado dos imóveis residenciais do índice ' + title_pmq, 
                 background = '#EEE')
    
    return(chart_pmq)

In [10]:
# Plotando gráfico com a função para o FipeZap geral
precometroquadrado(df_fz_vendas, 'FipeZap')

alt.Chart(...)

In [11]:
# Plotando gráfico com a função para o FipeZap de São Paulo
precometroquadrado(df_sp_vendas, 'FipeZap de São Paulo')

alt.Chart(...)

In [12]:
# Plotando gráfico com a função para o FipeZap do Rio de Janeiro
precometroquadrado(df_rj_vendas, 'FipeZap do Rio de Janeiro')

alt.Chart(...)

#### 9. Variação mensal do preço do metro quadrado dos imóveis residenciais.

In [13]:
# Copiando data frame do preço médio do m^2 para vendas
df_fz_retorno = df_fz_vendas.copy()
df_sp_retorno = df_sp_vendas.copy()
df_rj_retorno = df_rj_vendas.copy()

# Calculando retorno mensal
df_fz_retorno['Valorização do m^2'] = df_fz_vendas['Preço médio (R$/m^2)'] / df_fz_vendas['Preço médio (R$/m^2)'].shift(1) - 1
df_sp_retorno['Valorização do m^2'] = df_sp_vendas['Preço médio (R$/m^2)'] / df_sp_vendas['Preço médio (R$/m^2)'].shift(1) - 1
df_rj_retorno['Valorização do m^2'] = df_rj_vendas['Preço médio (R$/m^2)'] / df_rj_vendas['Preço médio (R$/m^2)'].shift(1) - 1

# Organizando indexes e pulando a primeira linha
df_fz_retorno = df_fz_retorno[['Data','Valorização do m^2']].iloc[1:,:].reset_index(drop = True)
df_sp_retorno = df_sp_retorno[['Data','Valorização do m^2']].iloc[1:,:].reset_index(drop = True)
df_rj_retorno = df_rj_retorno[['Data','Valorização do m^2']].iloc[1:,:].reset_index(drop = True)

# Função para construção do gráfico heatmap da variação mensal do preço do metro quadrado
def variacaomensalpreco(df_vmp, title_vmp):
    
    chart_vmp = alt.Chart(df_vmp).mark_rect().encode(
        alt.X('month(Data):O', title = 'Mês'),
        alt.Y('year(Data):O', title = 'Ano'),
        alt.Color('Valorização do m^2:Q', 
                  title = 'Variação', 
                  scale = alt.Scale(scheme = 'redblue', domain = [-0.04, 0.04]), 
                  legend = alt.Legend(format = '.0%'))
    ).properties(width = 640, height = 360, 
                 title = 'Variação mensal do preço do metro quadrado dos imóveis residenciais do índice ' + title_vmp)
    
    text_vmp = chart_vmp.mark_text(baseline = 'middle').encode(
        text = alt.Text('Valorização do m^2:Q', format = '.1%'),
        color = alt.value('#000000'))

    
    return(chart_vmp + text_vmp)

In [14]:
# Plotando gráfico com a função para o FipeZap geral
variacaomensalpreco(df_fz_retorno, 'FipeZap')

alt.LayerChart(...)

In [15]:
# Plotando gráfico com a função para o FipeZap de São Paulo
variacaomensalpreco(df_sp_retorno, 'FipeZap de São Paulo')

alt.LayerChart(...)

In [16]:
# Plotando gráfico com a função para o FipeZap do Rio de Janeiro
variacaomensalpreco(df_rj_retorno, 'FipeZap do Rio de Janeiro')

alt.LayerChart(...)

#### 10. Evolução do preço do metro quadrado para locação dos imóveis residenciais.

In [17]:
# Selecionando dados do preço médio do aluguel do m^2
df_fz_aluguel = df_fz.iloc[39:166, [1,37]]
df_sp_aluguel = df_sp.iloc[39:166, [1,37]]
df_rj_aluguel = df_rj.iloc[39:166, [1,37]]

# Renomeando colunas
df_fz_aluguel.columns = ['Data', 'Preço médio da locação (R$/m^2)']
df_sp_aluguel.columns = ['Data', 'Preço médio da locação (R$/m^2)']
df_rj_aluguel.columns = ['Data', 'Preço médio da locação (R$/m^2)']

# Convertendo coluna de datas para datetime
df_fz_aluguel['Data'] = pd.to_datetime(df_fz_aluguel['Data'])
df_sp_aluguel['Data'] = pd.to_datetime(df_sp_aluguel['Data'])
df_rj_aluguel['Data'] = pd.to_datetime(df_rj_aluguel['Data'])

# Organizando indexes
df_fz_aluguel = df_fz_aluguel.reset_index(drop = True)
df_sp_aluguel = df_sp_aluguel.reset_index(drop = True)
df_rj_aluguel = df_rj_aluguel.reset_index(drop = True)

# Função para construção do gráfico de linha do preço do metro quadrado para locação
def precometroquadradolocacao(df_pmql, title_pmql):
    
    chart_pmql = alt.Chart(df_pmql).mark_line().encode(
        x = 'Data:T',
        y = alt.Y('Preço médio da locação (R$/m^2)', scale = alt.Scale(domain = [0, 50])),
        color = alt.value('#110799')
    ).properties(width = 640, height = 360, 
                 title = 'Evolução do preço do metro quadrado para locação dos imóveis residenciais do índice ' + title_pmql,
                 background = '#EEE')
    
    return(chart_pmql)

In [18]:
# Plotando gráfico com a função para o FipeZap geral
precometroquadradolocacao(df_fz_aluguel, 'FipeZap')

alt.Chart(...)

In [19]:
# Plotando gráfico com a função para o FipeZap de São Paulo
precometroquadradolocacao(df_sp_aluguel, 'FipeZap de São Paulo')

alt.Chart(...)

In [20]:
# Plotando gráfico com a função para o FipeZap do Rio de Janeiro
precometroquadradolocacao(df_rj_aluguel, 'FipeZap do Rio de Janeiro')

alt.Chart(...)

#### 11. Yield anual da locação de imóveis residenciais.

In [21]:
# Concatenando coluna de data com coluna da rentabilidade do aluguel
df_fz_yield = pd.concat([df_fz.iloc[39:166, 1:2], df_fz.iloc[39:166, 42:43]], axis = 1)
df_sp_yield = pd.concat([df_sp.iloc[39:166, 1:2], df_sp.iloc[39:166, 42:43]], axis = 1)
df_rj_yield = pd.concat([df_rj.iloc[39:166, 1:2], df_rj.iloc[39:166, 42:43]], axis = 1)

# Renomeando colunas
df_fz_yield.columns = ['Data', 'Rental yield']
df_sp_yield.columns = ['Data', 'Rental yield']
df_rj_yield.columns = ['Data', 'Rental yield']

# Convertendo coluna de datas para datetime
df_fz_yield['Data'] = pd.to_datetime(df_fz_yield['Data'])
df_sp_yield['Data'] = pd.to_datetime(df_sp_yield['Data'])
df_rj_yield['Data'] = pd.to_datetime(df_rj_yield['Data'])

# Organizando indexes
df_fz_yield = df_fz_yield.reset_index(drop = True)
df_sp_yield = df_sp_yield.reset_index(drop = True)
df_rj_yield = df_rj_yield.reset_index(drop = True)

# Somando rentabilidade da locação anualmente
df_fz_yieldanual = df_fz_yield.groupby(pd.Grouper(key='Data', freq = 'Y')).sum().reset_index()
df_sp_yieldanual = df_sp_yield.groupby(pd.Grouper(key='Data', freq = 'Y')).sum().reset_index()
df_rj_yieldanual = df_rj_yield.groupby(pd.Grouper(key='Data', freq = 'Y')).sum().reset_index()

# Alterando mês final do ano de 2021
df_fz_yieldanual.loc[10, 'Data'] = pd.to_datetime('2021-07-31 00:00:00')
df_sp_yieldanual.loc[10, 'Data'] = pd.to_datetime('2021-07-31 00:00:00')
df_rj_yieldanual.loc[10, 'Data'] = pd.to_datetime('2021-07-31 00:00:00')

# Limitando a quantidade de casas depois da vírgula na coluna de yield
df_fz_yieldanual['Rental yield text'] = (round(df_fz_yieldanual['Rental yield'] * 100, 2)).astype(str) + '%'
df_sp_yieldanual['Rental yield text'] = (round(df_sp_yieldanual['Rental yield'] * 100, 2)).astype(str) + '%'
df_rj_yieldanual['Rental yield text'] = (round(df_rj_yieldanual['Rental yield'] * 100, 2)).astype(str) + '%'

# Função para construção do gráfico de barras do yield anual da locação dos imóveis residenciais
def yieldanuallocacao(df_yal, title_yal):
    
    # Média sem o valor quebrado de 2021
    df_yal['mean_yal'] = df_yal.iloc[:10,:]['Rental yield'].mean()
    
    chart_yal = alt.Chart(df_yal).mark_bar().encode(
        x = alt.X('yearmonth(Data):O', title = 'Ano'),
        y = alt.Y('Rental yield:Q', title = 'Rental yield e média', axis = alt.Axis(format = '%'), 
                  scale = alt.Scale(domain = [0, 0.066])),
        color = alt.value('#110799')
    ).properties(width = 640, height = 360, title = 'Yield anual da locação de imóveis residenciais do índice ' + title_yal)
    
    line_yal = alt.Chart(df_yal).mark_rule().encode(
    y = alt.Y('mean(mean_yal):Q', title = ''),
    color = alt.value('#8F9907'))
    
    text_yal = chart_yal.mark_text(baseline = 'bottom').encode(
        text = alt.Text('Rental yield text:N'),
        color = alt.value('#000000'))
    
    return((chart_yal + line_yal + text_yal).configure(background = '#EEE'))

In [22]:
# Plotando gráfico com a função para o FipeZap geral
yieldanuallocacao(df_fz_yieldanual, 'FipeZap')

alt.LayerChart(...)

In [23]:
# Plotando gráfico com a função para o FipeZap de São Paulo
yieldanuallocacao(df_sp_yieldanual, 'FipeZap de São Paulo')

alt.LayerChart(...)

In [24]:
# Plotando gráfico com a função para o FipeZap do Rio de Janeiro
yieldanuallocacao(df_rj_yieldanual, 'FipeZap do Rio de Janeiro')

alt.LayerChart(...)

#### 12. Variação acumulada e comparação com o IPCA.

In [25]:
# Organizando dados com seleção, transposição e exclusão os indexes
df_ipca = df_ipca.iloc[[0,2], 200:327].T.reset_index(drop = True)

# Renomeando colunas
df_ipca.columns = ['Data', 'IPCA']

# Substituindo coluna de datas
df_ipca['Data'] = df_ifix_retorno['Data']

# Alterando tipo da coluna do índice
df_ipca['IPCA'] = df_ipca['IPCA'].str.replace(',', '.').astype(float)

# Alterando valores percentuais
df_ipca['IPCA'] = df_ipca['IPCA'] / 100

# Criando uma cópia do data frame dos valores do IPCA
df_retornoacumulado = df_ipca.copy()

# Calculando valores acumulados do ipca
for i in range(0, 127):
    df_retornoacumulado.loc[i, 'IPCA2'] = (df_retornoacumulado.iloc[0:i+1, 1] + 1).product() - 1
    
# Agrupando retorno acumulado do IFIX
for i in range(0, 127):
    df_retornoacumulado.loc[i, 'IFIX'] = (df_ifix_retorno.iloc[0:i+1, 1] + 1).product() - 1
    
# Agrupando retorno acumulado do FipeZap
for i in range(0, 127):
    df_retornoacumulado.loc[i, 'FipeZap'] = (df_fz_vendas.loc[i+1, 'Preço médio (R$/m^2)'] + df_fz_aluguel.iloc[0:i+1, 1].sum()) / df_fz_vendas.loc[0, 'Preço médio (R$/m^2)'] - 1
    
# Agrupando retorno acumulado do FipeZap em São Paulo
for i in range(0, 127):
    df_retornoacumulado.loc[i, 'FipeZap-SP'] = (df_sp_vendas.loc[i+1, 'Preço médio (R$/m^2)'] + df_sp_aluguel.iloc[0:i+1, 1].sum()) / df_sp_vendas.loc[0, 'Preço médio (R$/m^2)'] - 1
    
# Agrupando retorno acumulado do FipeZap no Rio de Janeiro
for i in range(0, 127):
    df_retornoacumulado.loc[i, 'FipeZap-RJ'] = (df_rj_vendas.loc[i+1, 'Preço médio (R$/m^2)'] + df_rj_aluguel.iloc[0:i+1, 1].sum()) / df_rj_vendas.loc[0, 'Preço médio (R$/m^2)'] - 1
    
# Apagando coluna do IPCA mensal
df_retornoacumulado = df_retornoacumulado.drop(['IPCA'], axis = 1)

# Trocando nome da coluna ipca2
df_retornoacumulado = df_retornoacumulado.rename(columns = {'IPCA2': 'IPCA'})

In [26]:
# Plotando gráfico de linhas com o compativo entre os retornos acumulados do FipeZap geral, IFIX e IPCA
alt.Chart(df_retornoacumulado[['Data', 'FipeZap', 'IPCA', 'IFIX']].melt(
    'Data', var_name = 'Índice', value_name = 'Retorno acumulado')).mark_line().encode(
    x = 'Data:T',
    y = alt.Y('Retorno acumulado:Q', axis = alt.Axis(format = '%'), scale = alt.Scale(domain = [0, 2.40])),
    color = alt.Color('Índice:N',
    scale=alt.Scale(domain = ['FipeZap', 'IFIX', 'IPCA'], 
                    range = ['#110799', '#991107', '#079911']))
).properties(width = 640, height = 360, 
             title = 'Retorno acumulado do FipeZap, IFIX e IPCA',
             background = '#EEE')

alt.Chart(...)

In [27]:
# Plotando gráfico de linhas com o compativo entre os retornos acumulados do FipeZap (SP e RJ), IFIX e IPCA
alt.Chart(df_retornoacumulado[['Data', 'FipeZap-SP', 'FipeZap-RJ', 'IPCA', 'IFIX']].melt(
    'Data', var_name = 'Índice', value_name = 'Retorno acumulado')).mark_line().encode(
    x = 'Data:T',
    y = alt.Y('Retorno acumulado:Q', axis = alt.Axis(format = '%'), scale = alt.Scale(domain = [0, 2.40])),
    color = alt.Color('Índice:N',
    scale=alt.Scale(domain = ['FipeZap-RJ', 'FipeZap-SP', 'IFIX', 'IPCA'], 
                    range = ['#110799', '#990746', '#8F9907', '#07995A']))
).properties(width = 640, height = 360, 
             title = 'Retorno acumulado do FipeZap-RJ, FipeZap-SP, IFIX e IPCA',
             background = '#EEE')

alt.Chart(...)

In [28]:
# Inflação total e ao ano no período
print(round(100 * df_retornoacumulado.loc[126, 'IPCA'], 2).astype(str) + '%' + ' de inflação total')
print(round(100 * ((1 + df_retornoacumulado.loc[126, 'IPCA']) ** (12/127) - 1), 2).astype(str) + '%' + ' de inflação ao ano')

82.28% de inflação total
5.84% de inflação ao ano


In [29]:
# Retorno acumulado total (nominal)
for i in ['IFIX', 'FipeZap', 'FipeZap-RJ', 'FipeZap-SP']:
    print(round(100 * df_retornoacumulado.loc[126, i], 2).astype(str) + '%' + ' ' + i)

182.41% IFIX
173.49% FipeZap
149.48% FipeZap-RJ
206.64% FipeZap-SP


In [30]:
# Retorno acumulado ao ano (nominal)
for i in ['IFIX', 'FipeZap', 'FipeZap-RJ', 'FipeZap-SP']:
    print(round(100 * ((1 + df_retornoacumulado.loc[126, i]) ** (12/127) - 1), 2).astype(str) + '%' + ' ' + i)

10.31% IFIX
9.97% FipeZap
9.02% FipeZap-RJ
11.17% FipeZap-SP


In [31]:
# Retorno acumulado total (real)
for i in ['IFIX', 'FipeZap', 'FipeZap-RJ', 'FipeZap-SP']:
    rr_var = (1 + df_retornoacumulado.loc[126, i])/(1 + df_retornoacumulado.loc[126, 'IPCA'])
    print(round(100 * (rr_var - 1), 2).astype(str) + '%' + ' ' + i)

54.94% IFIX
50.04% FipeZap
36.87% FipeZap-RJ
68.23% FipeZap-SP


In [32]:
# Retorno acumulado ao ano (real)
for i in ['IFIX', 'FipeZap', 'FipeZap-RJ', 'FipeZap-SP']:
    rra_var = ((1 + df_retornoacumulado.loc[126, i])/(1 + df_retornoacumulado.loc[126, 'IPCA'])) ** (12/127)
    print(round(100 * (rra_var - 1), 2).astype(str) + '%' + ' ' + i)

4.22% IFIX
3.91% FipeZap
3.01% FipeZap-RJ
5.04% FipeZap-SP


#### 13. Desempenho histórico do S&P500.

In [33]:
# Selecionando colunas necessárias
df_sp500 = df_sp500[['Data', 'Fechamento']]

# Convertendo coluna de datas para datetime
df_sp500['Data'] = df_ifix['Data']

# Plotando gráfico de linha do desempenho do S&P500 no período
alt.Chart(df_sp500).mark_line().encode(
    x = 'Data:T',
    y = alt.Y('Fechamento:Q', title = 'Valor'),
    color = alt.value('#110799')
).properties(width = 640, height = 360, 
             title = 'Desempenho histórico do S&P500',
             background = '#EEE')

alt.Chart(...)

#### 14. Cálculo dos betas.
$\beta_i = \dfrac{cov(r_i, r_m)}{var(r_m)}$

In [34]:
# Copiando data frame dos valores absolutos
df_sp500_retorno = df_sp500.copy()

# Calculando retorno do índice
df_sp500_retorno['Variação'] = df_sp500_retorno['Fechamento'] / df_sp500_retorno['Fechamento'].shift(1) - 1

# Recortando primeira linha e coluna de valores do índice
df_sp500_retorno = df_sp500_retorno.iloc[1:, [0, 2]].reset_index(drop = True)

# Função para fragmentar rentabilidade acumulada calculada anteriormente
def function_rm(var):

    df_function_rm = df_retornoacumulado.copy()
    
    df_function_rm.loc[0, 'Variação'] = df_function_rm.loc[0, var]

    for i in range(1, 127):
        df_function_rm.loc[i, 'Variação'] = (1 + df_function_rm.loc[i, var]) / (1 + df_function_rm.loc[i-1, var]) - 1
        
    return(df_function_rm[['Data', 'Variação']])

In [35]:
(df_sp500_retorno['Variação']).var()

0.0014715691172151844

In [36]:
# Beta do IFIX
beta_ifix = cov(function_rm('IFIX')['Variação'], df_sp500_retorno['Variação'])[0, 1] / (df_sp500_retorno['Variação']).var()
round(beta_ifix, 2)

0.28

In [37]:
# Beta do FIPEZAP
beta_fz = cov(function_rm('FipeZap')['Variação'], df_sp500_retorno['Variação'])[0, 1] / (df_sp500_retorno['Variação']).var()
round(beta_fz, 3)

-0.003

In [38]:
# Beta do FIPEZAP-SP
beta_fzsp = cov(function_rm('FipeZap-SP')['Variação'], df_sp500_retorno['Variação'])[0, 1] / (df_sp500_retorno['Variação']).var()
round(beta_fzsp, 3)

-0.004

In [39]:
# Beta do FIPEZAP-RJ
beta_fzrj = cov(function_rm('FipeZap-RJ')['Variação'], df_sp500_retorno['Variação'])[0, 1] / (df_sp500_retorno['Variação']).var()
round(beta_fzrj, 2)

-0.01

#### 15. Evolução do risco-país (EMBI+).

In [40]:
# Alterando risco-país para valores percentuais
df_embi['Risco-país'] = df_embi['Risco-país'] / 10000

# Plotando gráfico de linha com os dados do risco-país EMBI+
base = alt.Chart(df_embi).mark_line().encode(
    x = alt.X('Data:T', title = 'Data'),
    y = alt.Y('Risco-país:Q', title = 'Valor percentual e mediana', axis = alt.Axis(format = '%')),
    color = alt.value('#110799')
).properties(width = 640, height = 360, 
             title = 'Evolução do risco-país (EMBI+)')

line = alt.Chart(df_embi).mark_rule().encode(
    y = alt.Y('median(Risco-país):Q', title = ''),
    color = alt.value('#8F9907'))

(base + line).configure(background = '#EEE')

alt.LayerChart(...)

#### 16. Evolução da rentabilidade do título americano real de dez anos.

In [41]:
# Selecionando colunas 
df_t10y = df_t10y[['Data', 'Fechamento']] 

# Convertendo coluna de fechamento para valores percentuais
df_t10y['Fechamento'] = df_t10y['Fechamento'].astype(float)/100

# Plotando gráfico de linha com os dados do título americano de dez anos
base = alt.Chart(df_t10y).mark_line().encode(
    x = alt.X('Data:T', title = 'Data'),
    y = alt.Y('Fechamento:Q', title = 'Real Treasury 10 years e mediana', axis = alt.Axis(format = '%'), 
              scale = alt.Scale(domain = [-0.015, 0.015])),
    color = alt.value('#110799')
).properties(width = 640, height = 360, 
             title = 'Evolução da rentabilidade do título americano real de dez anos')

line = alt.Chart(df_t10y).mark_rule().encode(
    y = alt.Y('median(Fechamento):Q', title = ''),
    color = alt.value('#8F9907'))

(base + line).configure(background = '#EEE')

alt.LayerChart(...)

#### 17. Cálculo do retorno esperado dos investimentos.

$E(R_i) = R_f + \beta_i \cdot [E(R_m) - R_f] + r_{Brasil}$

In [42]:
# Risk free real USA (mediana): 0.00305
r_f = df_t10y['Fechamento'].median()

# Prêmio de mercado histórico
r_m = 0.06

# Mediana do risco-país (EMBI+): 0.0252
r_p = df_embi['Risco-país'].median() 

In [43]:
# IFIX
e_ifix = r_f + beta_ifix * (r_m - r_f) + r_p
print(round(e_ifix * 100, 2).astype(str) + '%')

4.44%


In [44]:
# FipeZap
e_fz = r_f + beta_fz * (r_m - r_f) + r_p
print(round(e_fz * 100, 2).astype(str) + '%')

2.85%


In [45]:
# FipeZap-SP
e_fzsp = r_f + beta_fzsp * (r_m - r_f) + r_p
print(round(e_fzsp * 100, 2).astype(str) + '%')

2.85%


In [46]:
# FipeZap-RJ
e_fzrj = r_f + beta_fzrj * (r_m - r_f) + r_p
print(round(e_fzrj * 100, 2).astype(str) + '%')

2.82%


#### 18. Cálculo do índice de sharpe.

$IS_i = \dfrac{E(R_i) - R_f}{\sigma_i}$

In [47]:
# IFIX
round((e_ifix - r_f) / ((function_rm('IFIX')['Variação']).std() * 12 ** 0.5), 2)

0.37

In [48]:
# FipeZap
round((e_fz - r_f) / ((function_rm('FipeZap')['Variação']).std() * 12 ** 0.5), 2)

1.06

In [49]:
# FipeZap-SP
round((e_fzsp - r_f) / ((function_rm('FipeZap-SP')['Variação']).std() * 12 ** 0.5), 2)

1.05

In [50]:
# FipeZap-RJ
round((e_fzrj - r_f) / ((function_rm('FipeZap-RJ')['Variação']).std() * 12 ** 0.5), 2)

0.77

#### 19. Teste de significância do beta.

In [51]:
# Função para rodar a regressão linear dos dados
def regressao(var):
    
    y = function_rm(var)['Variação']
    X = df_sp500_retorno['Variação']

    X_sm = sm.add_constant(X)
    results = sm.OLS(y, X_sm).fit()
    
    return(results.summary())

In [52]:
# Regressão do IFIX 
regressao('IFIX')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Variação   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     15.79
Date:                Fri, 08 Oct 2021   Prob (F-statistic):           0.000119
Time:                        14:21:13   Log-Likelihood:                 265.56
No. Observations:                 127   AIC:                            -527.1
Df Residuals:                     125   BIC:                            -521.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0058      0.003      2.076      0.040       0.000       0.011
Variação       0.2782      0.070      3.974      0.000       0.140       0.417
==============================================================================
Omnibus:                       15.940   Durbin-Watson:                   1.572
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.701
Skew:                          -0.469   Prob(JB):                     2.92e-08
Kurtosis:                       5.383   Cond. No.                         26.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
# Regressão do FipeZap
regressao('FipeZap')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Variação   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                   0.04082
Date:                Fri, 08 Oct 2021   Prob (F-statistic):              0.840
Time:                        14:21:13   Log-Likelihood:                 453.74
No. Observations:                 127   AIC:                            -903.5
Df Residuals:                     125   BIC:                            -897.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0080      0.001     12.699      0.000       0.007       0.009
Variação      -0.0032      0.016     -0.202      0.840      -0.035       0.028
==============================================================================
Omnibus:                       40.277   Durbin-Watson:                   0.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.133
Skew:                           1.532   Prob(JB):                     2.64e-15
Kurtosis:                       4.816   Cond. No.                         26.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
# Regressão do FipeZap de São Paulo
regressao('FipeZap-SP')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Variação   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                   0.06454
Date:                Fri, 08 Oct 2021   Prob (F-statistic):              0.800
Time:                        14:21:13   Log-Likelihood:                 452.82
No. Observations:                 127   AIC:                            -901.6
Df Residuals:                     125   BIC:                            -895.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0089      0.001     14.052      0.000       0.008       0.010
Variação      -0.0041      0.016     -0.254      0.800      -0.036       0.028
==============================================================================
Omnibus:                       35.125   Durbin-Watson:                   0.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.975
Skew:                           1.439   Prob(JB):                     3.14e-12
Kurtosis:                       4.314   Cond. No.                         26.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [55]:
# Regressão do FipeZap do Rio de Janeiro
regressao('FipeZap-RJ')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Variação   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1960
Date:                Fri, 08 Oct 2021   Prob (F-statistic):              0.659
Time:                        14:21:14   Log-Likelihood:                 414.99
No. Observations:                 127   AIC:                            -826.0
Df Residuals:                     125   BIC:                            -820.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0074      0.001      8.609      0.000       0.006       0.009
Variação      -0.0096      0.022     -0.443      0.659      -0.052       0.033
==============================================================================
Omnibus:                       38.133   Durbin-Watson:                   0.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.129
Skew:                           1.483   Prob(JB):                     5.32e-14
Kurtosis:                       4.659   Cond. No.                         26.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 20. CDI.

In [56]:
# Selecionando colunas 
df_cdi = df_cdi[['Data', 'Fechamento']] 

# Convertendo coluna de fechamento para valores percentuais
df_cdi['Fechamento'] = df_cdi['Fechamento'].astype(float)/100

# Plotando gráfico de linha com os dados do cdi
base = alt.Chart(df_cdi).mark_line().encode(
    x = alt.X('Data:T', title = 'Data'),
    y = alt.Y('Fechamento:Q', title = 'CDI e mediana', axis = alt.Axis(format = '%'), 
              scale = alt.Scale(domain = [0, 0.15])),
    color = alt.value('#110799')
).properties(width = 640, height = 360, 
             title = 'Evolução da rentabilidade do CDI')

line = alt.Chart(df_cdi).mark_rule().encode(
    y = alt.Y('median(Fechamento):Q', title = ''),
    color = alt.value('#8F9907'))

(base + line).configure(background = '#EEE')

alt.LayerChart(...)

#### 21. Cálculo do indice de sharpe no período.

In [57]:
# Mediana d taxa livre de risco (cdi): 0,0873
r_f2 = df_cdi['Fechamento'].median()

In [58]:
# IFIX
exr_ifix = (((1 + df_retornoacumulado.loc[126, 'IFIX']) ** (12/127) - 1) - r_f2) 
round(exr_ifix / ((function_rm('IFIX')['Variação']).std() * 12 ** 0.5), 2)

0.14

In [59]:
# FipeZap
exr_fz = (((1 + df_retornoacumulado.loc[126, 'FipeZap']) ** (12/127) - 1) - r_f2) 
round(exr_fz / ((function_rm('FipeZap')['Variação']).std() * 12 ** 0.5), 2)

0.53

In [60]:
# FipeZap-SP
exr_fzsp = (((1 + df_retornoacumulado.loc[126, 'FipeZap-SP']) ** (12/127) - 1) - r_f2) 
round(exr_fzsp / ((function_rm('FipeZap-SP')['Variação']).std() * 12 ** 0.5), 2)

1.02

In [61]:
# FipeZap-RJ
exr_fzrj = (((1 + df_retornoacumulado.loc[126, 'FipeZap-RJ']) ** (12/127) - 1) - r_f2) 
round(exr_fzrj / ((function_rm('FipeZap-RJ')['Variação']).std() * 12 ** 0.5), 2)

0.09

#### 21. Montagem e cálculos do SUNO30.

In [62]:
# Importando bibliotecas
import pandas as pd

# Criando data frame vazio
df_concat = pd.DataFrame()

# Concatenando preços dos fundos imobiliários do SUNO30
for i in ['HGBS11', 'BBPO11', 'BRCR11', 'BTCR11', 'GGRC11', 'HGCR11', 'HGLG11', 'HGRE11', 'HGRU11', 'IRDM11', 'XPLG11', 'JSRE11', 'KNCR11', 'KNHY11', 'KNIP11', 'KNRI11', 'LVBI11', 'MALL11', 'MXRF11', 'RBRR11', 'RBVA11', 'RCRB11', 'RECT11', 'SDIL11', 'UBSR11', 'VILG11', 'VISC11', 'VRTA11', 'XPIN11', 'XPML11']:
    
    df = (pd.read_excel('FIIsSUNO30/{}.xlsx'.format(i), skiprows = 3).set_index('Data')[['Fechamento']])
    
    df.columns = ['{}'.format(i)]
    
    df_concat = pd.concat([df_concat, df], axis = 1)
    
# Calculando retorno mensal 
for j in range(0, 30):
    
    df_concat.iloc[:, j:j+1] = df_concat.iloc[:, j:j+1].astype(float) / df_concat.iloc[:, j:j+1].shift(1).astype(float) - 1

# Eliminando primeira linha
df_concat = df_concat.iloc[1:, :]

# Resetando indexes
df_concat = df_concat.reset_index(drop = True)

# Calculando variação mensal
for k in range(0, 127):
    df_concat.loc[k, 'Variação mensal'] = df_concat.iloc[k:k+1, :].mean().mean()
    
# Calculando retorno acumulado
for l in range(0, 127):
    df_concat.loc[l, 'SUNO30'] = (1 + df_concat.iloc[:l+1, 30:]['Variação mensal']).product() - 1
    
# Adicionando coluna das datas
df_concat = pd.concat([df_concat, df_ifix_retorno['Data']], axis = 1)
    
# Criando data frame somente com a variação mensal e retorno acumulado
df_suno30 = df_concat[['Data','Variação mensal', 'SUNO30']]

# Plotando gráfico de linhas com o compativo entre os retornos acumulados do FipeZap, IFIX, IPCA e SUNO30
alt.Chart(pd.concat((df_suno30[['Data', 'SUNO30']], df_retornoacumulado[['FipeZap', 'IPCA', 'IFIX']]), axis = 1).melt(
    'Data', var_name = 'Índice', value_name = 'Retorno acumulado')).mark_line().encode(
    x = 'Data:T',
    y = alt.Y('Retorno acumulado:Q', axis = alt.Axis(format = '%'), scale = alt.Scale(domain = [0, 3.5])),
    color = alt.Color('Índice:N',
    scale=alt.Scale(domain = ['FipeZap', 'IFIX', 'IPCA', 'SUNO30'], 
                    range = ['#110799', '#990746', '#8F9907', '#07995A']))
).properties(width = 640, height = 360, 
             title = 'Retorno acumulado do FipeZap, IFIX, IPCA e SUNO30',
             background = '#EEE')

alt.Chart(...)

In [63]:
# Plotando gráfico heatmap da variação ao longo do tempo
base = alt.Chart(df_suno30).mark_rect().encode(
    alt.X('month(Data):O', title = 'Mês'),
    alt.Y('year(Data):O', title = 'Ano'),
    alt.Color('Variação mensal:Q', title = 'Variação', scale = alt.Scale(scheme = 'redblue', domain = [-0.2, 0.2]), 
              legend = alt.Legend(format = '.0%'))
).properties(width = 640, height = 360, title = 'Variação mensal do SUNO30')

text = base.mark_text(baseline = 'middle').encode(
    text = alt.Text('Variação mensal:Q', format = '.1%'),
    color = alt.value('#000000'))

base + text

alt.LayerChart(...)

In [64]:
# Retorno real acumulado do SUNO30
print(round(((1 + df_suno30.loc[126, 'SUNO30'])/(1 + df_retornoacumulado.loc[126, 'IPCA']) - 1) * 100, 2).astype(str) + '%')

109.51%


In [65]:
# Retorno real ao ano
print(round((((1 + df_suno30.loc[126, 'SUNO30'])/(1 + df_retornoacumulado.loc[126, 'IPCA'])) ** (12/127) - 1) * 100, 2).astype(str) + '%')

7.24%


In [66]:
# Índice de Sharpe do SUNO30
exr_suno30 = (((1 + df_suno30.loc[126, 'SUNO30']) ** (12/127) - 1) - r_f2)
round(exr_suno30 / ((df_suno30['Variação mensal']).std() * 12 ** 0.5), 2)

0.42